# Notebook Title

## Setup Python and R environment
you can ignore this section

In [2]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [3]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [4]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## 👉 download your data

You can write code here to download your dataset. Or if you already have it, just leave the URL in the comments and just load it into a pandas or R (or both) dataframe.

In [5]:
import pandas as pd

df = pd.read_csv('schools.csv')
df.dtypes

Borough              object
Community Board       int64
School Type          object
School Name / ID     object
Address              object
Postcode              int64
Latitude            float64
Longitude           float64
Council District      int64
Census Tract          int64
BIN                   int64
BBL                   int64
NTA                  object
dtype: object

## 👉 convert addresses --> lat/long 

See the [census-examples](https://github.com/data4news/census-examples) repository for examples. If you need help, try asking in the class slack channel. Chances are someone in the class is struggling with the same problem as you are so we might as well all learn together in the same slack channel! 

## 👉 convert lat/long to census geography codes 

(like 'GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK', etc...)

Same note as above, see [census-examples](https://github.com/data4news/census-examples) repository for examples or ask in the class slack channel if stuck.

In [6]:
# Code adapted from:
# https://gis.stackexchange.com/questions/363830/applying-the-censusgeocode-package-to-an-entire-dataframe-of-geocoded-data
# Defines a geocode function that accepts lat/long and spits out geographies
# The code then runs that funciton in parllel (for speed).

import pandas as pd
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import glob
import json
import requests
import pandas as pd
from pprint import pprint
from tqdm import tqdm


import requests_cache
cache = requests_cache.CachedSession("geocode_cache", backend="filesystem")

def geocode(lat, lng):
    try:
        url = "https://geocoding.geo.census.gov/geocoder/geographies/coordinates"
        params = {
            "x": lng,
            "y": lat,
            "benchmark": "Public_AR_Census2020",
            "vintage": "Census2020_Census2020",
            "format": "json"
        }
        response = cache.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        census = data['result']['geographies']['Census Blocks'][0]
        return census
    except Exception as e:
        print(f"Error geocoding ({lat}, {lng}): {e}")
        return None

def bulk_geocode(latitudes, longitudes):
    """
    Geocode a list of latitudes and longitudes in parallel (for speed).
    """

    with ThreadPoolExecutor() as tpe:
        latitudes = df['Latitude']
        longitudes = df['Longitude']
        mapped_results = tpe.map(geocode, latitudes, longitudes)
        data = list(tqdm(mapped_results, total=len(df)))

    return pd.DataFrame(data)

census_geos_df = bulk_geocode(df['Latitude'], df['Longitude']) 
census_geos_df.head()


100%|██████████| 135/135 [00:00<00:00, 1233.69it/s]


,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,TRACT,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY
0,,0,360050075002004,+40.8195549,2004,0,36,2004,210701006029095,BK,...,007500,-073.9089637,2,13831,0,-073.9089637,G5040,L,U,005
1,,346,360050087001001,+40.8200146,1001,0,36,1001,210701006017935,BK,...,008700,-073.8984541,1,20125,149,-073.8984541,G5040,L,U,005
2,,545,360050145003000,+40.8320453,3000,0,36,3000,210701006028952,BK,...,014500,-073.9059143,3,17387,179,-073.9059143,G5040,L,U,005
3,,618,360050145003001,+40.8324032,3001,0,36,3001,210701006027194,BK,...,014500,-073.9069663,3,21560,207,-073.9069663,G5040,L,U,005
4,,290,360050185024003,+40.8228807,4003,0,36,4003,210701006029362,BK,...,018502,-073.9075830,4,16696,113,-073.9075830,G5040,L,U,005


In [7]:
census_geos_df['GEOID_Modified'] = census_geos_df['GEOID'].astype(str).str.slice(0, -4)

# Display the updated dataframe to verify the new column
census_geos_df.head(20)


,SUFFIX,POP100,GEOID,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY,GEOID_Modified
0,,0,360050075002004,+40.8195549,2004,0,36,2004,210701006029095,BK,...,-073.9089637,2,13831,0,-073.9089637,G5040,L,U,005,36005007500
1,,346,360050087001001,+40.8200146,1001,0,36,1001,210701006017935,BK,...,-073.8984541,1,20125,149,-073.8984541,G5040,L,U,005,36005008700
2,,545,360050145003000,+40.8320453,3000,0,36,3000,210701006028952,BK,...,-073.9059143,3,17387,179,-073.9059143,G5040,L,U,005,36005014500
3,,618,360050145003001,+40.8324032,3001,0,36,3001,210701006027194,BK,...,-073.9069663,3,21560,207,-073.9069663,G5040,L,U,005,36005014500
4,,290,360050185024003,+40.8228807,4003,0,36,4003,210701006029362,BK,...,-073.9075830,4,16696,113,-073.9075830,G5040,L,U,005,36005018502
5,,0,360050219001005,+40.8394289,1005,0,36,1005,210701006027480,BK,...,-073.9190205,1,23105,0,-073.9190205,G5040,L,U,005,36005021900
6,,739,360050221023002,+40.8364830,3002,0,36,3002,210701006017492,BK,...,-073.9172515,3,14668,215,-073.9172515,G5040,L,U,005,36005022102
7,,5,360050183022000,+40.8279874,2000,0,36,2000,210701006029968,BK,...,-073.9176803,2,12099,0,-073.9176803,G5040,L,U,005,36005018302
8,,266,360050223002001,+40.8402608,2001,0,36,2001,210701006027345,BK,...,-073.9159782,2,10953,78,-073.9159782,G5040,L,U,005,36005022300
9,,908,360050179021001,+40.8368571,1001,0,36,1001,210701006017347,BK,...,-073.9135049,1,20718,277,-073.9135049,G5040,L,U,005,36005017902


In [8]:
census_geos_df.rename(columns={'GEOID': 'GEOID_long', 'GEOID_Modified': 'GEOID'}, inplace=True)
census_geos_df.head(20)

,SUFFIX,POP100,GEOID_long,CENTLAT,BLOCK,AREAWATER,STATE,BASENAME,OID,LSADC,...,CENTLON,BLKGRP,AREALAND,HU100,INTPTLON,MTFCC,LWBLKTYP,UR,COUNTY,GEOID
0,,0,360050075002004,+40.8195549,2004,0,36,2004,210701006029095,BK,...,-073.9089637,2,13831,0,-073.9089637,G5040,L,U,005,36005007500
1,,346,360050087001001,+40.8200146,1001,0,36,1001,210701006017935,BK,...,-073.8984541,1,20125,149,-073.8984541,G5040,L,U,005,36005008700
2,,545,360050145003000,+40.8320453,3000,0,36,3000,210701006028952,BK,...,-073.9059143,3,17387,179,-073.9059143,G5040,L,U,005,36005014500
3,,618,360050145003001,+40.8324032,3001,0,36,3001,210701006027194,BK,...,-073.9069663,3,21560,207,-073.9069663,G5040,L,U,005,36005014500
4,,290,360050185024003,+40.8228807,4003,0,36,4003,210701006029362,BK,...,-073.9075830,4,16696,113,-073.9075830,G5040,L,U,005,36005018502
5,,0,360050219001005,+40.8394289,1005,0,36,1005,210701006027480,BK,...,-073.9190205,1,23105,0,-073.9190205,G5040,L,U,005,36005021900
6,,739,360050221023002,+40.8364830,3002,0,36,3002,210701006017492,BK,...,-073.9172515,3,14668,215,-073.9172515,G5040,L,U,005,36005022102
7,,5,360050183022000,+40.8279874,2000,0,36,2000,210701006029968,BK,...,-073.9176803,2,12099,0,-073.9176803,G5040,L,U,005,36005018302
8,,266,360050223002001,+40.8402608,2001,0,36,2001,210701006027345,BK,...,-073.9159782,2,10953,78,-073.9159782,G5040,L,U,005,36005022300
9,,908,360050179021001,+40.8368571,1001,0,36,1001,210701006017347,BK,...,-073.9135049,1,20718,277,-073.9135049,G5040,L,U,005,36005017902


## 👉 Output Data

Output your dataframe containing your data and the Census connector codes (like tract, block, etc...).

In [9]:
to_keep = ['GEOID', 'STATE', 'COUNTY', 'TRACT', 'BLOCK']
census_geos_df = census_geos_df[to_keep]
census_geos_df

,GEOID,STATE,COUNTY,TRACT,BLOCK
0,36005007500,36,005,007500,2004
1,36005008700,36,005,008700,1001
2,36005014500,36,005,014500,3000
3,36005014500,36,005,014500,3001
4,36005018502,36,005,018502,4003
...,...,...,...,...,...
130,36061018000,36,061,018000,1002
131,36061026900,36,061,026900,3000
132,36061025100,36,061,025100,2001
133,36061024500,36,061,024500,6001


In [10]:
df_with_geos = pd.concat(
    [ 
        df.reset_index(drop=True),
        census_geos_df.reset_index(drop=True)
    ], 
    axis=1)

df_with_geos.head()

,Borough,Community Board,School Type,School Name / ID,Address,Postcode,Latitude,Longitude,Council District,Census Tract,BIN,BBL,NTA,GEOID,STATE,COUNTY,TRACT,BLOCK
0,Bronx,1,Public,P.S. 157 GROVE HILL,757 Caudwell Avenue,10456,40.819258,-73.908553,17,75,2004464,2026250001,Melrose South-Mott Haven North,36005007500,36,005,007500,2004
1,Bronx,2,Public,P.S. 60 (New School #2),888 Rev James A. Polite Avenue,10459,40.820403,-73.898912,17,87,2005348,2026960085,Longwood ...,36005008700,36,005,008700,1001
2,Bronx,3,Public,HARRIET TUBMAN CHARTER SCHOOL,3565 3rd Avenue,10456,40.832010,-73.905392,16,145,2001331,2023730014,Claremont-Bathgate ...,36005014500,36,005,014500,3000
3,Bronx,3,Public,P.S. 132 GARRETT A. MORGAN SCHOOL,1245 Washington Avenue,10456,40.831827,-73.906607,16,145,2001619,2023900001,Claremont-Bathgate ...,36005014500,36,005,014500,3001
4,Bronx,3,Public,P.S. 140 EAGLE SCHOOL,916 Eagle Avenue,10456,40.823289,-73.907948,17,185,2004469,2026270009,Morrisania-Melrose ...,36005018502,36,005,018502,4003


In [11]:
df_with_geos.to_csv('geocoded_schools.csv', index=False)
